In [1]:
import csv
import math
import pandas as pd
import numpy as np

In [2]:
def std_dev(distr, avg, num_samples):
    sum_of_squares = 0
    for k,v in distr.items():
        sum_of_squares += v*((k-avg)**2)
    variance = sum_of_squares/num_samples
    return math.sqrt(variance)

In [3]:
training_df = pd.read_csv('training.csv')
training_df.drop('article_number', axis=1, inplace=True)
print(training_df)

                                          article_words  \
0     open,absent,cent,cent,cent,stock,inflow,rate,k...   
1     morn,stead,end,end,day,day,day,patch,patch,pat...   
2     socc,socc,world,world,recent,law,fifa,fifa,fif...   
3     open,forint,forint,forint,forint,cent,cent,ste...   
4     morn,complet,weekend,minut,minut,minut,arrow,d...   
5     regist,equal,stock,stock,city,city,period,issu...   
6     cumulat,record,month,bank,balanc,cash,jan,jan,...   
7     divid,ten,rule,weekend,social,sweep,destruc,su...   
8     hit,regist,publ,gmt,count,week,fall,articl,art...   
9     queen,queen,park,hit,minist,minist,minist,mini...   
10    contrast,research,heavy,heavy,weather,sud,mirr...   
11    weather,underl,half,half,half,half,half,half,h...   
12    portug,portug,rate,fix,money,escudo,drain,drai...   
13    open,shekel,shekel,shekel,shekel,shekel,shekel...   
14    mgt,conc,praecip,limit,emtn,bp,year,full,und,m...   
15    stabl,stabl,strength,cent,cent,cent,hagstrom,q... 

In [4]:
labels = training_df['topic']
label_distr = labels.value_counts()
print(label_distr)
print(label_distr.shape)
print(label_distr['IRRELEVANT'])
label_sum = label_distr.sum()
print(label_sum)
label_distr = pd.concat([pd.Series(data=[label_sum], index=['global']), label_distr])
print(label_distr)
print(label_distr['DEFENCE'])

IRRELEVANT                          4734
MONEY MARKETS                       1673
SPORTS                              1102
FOREX MARKETS                        845
DEFENCE                              258
SHARE LISTINGS                       218
HEALTH                               183
BIOGRAPHIES PERSONALITIES PEOPLE     167
DOMESTIC MARKETS                     133
ARTS CULTURE ENTERTAINMENT           117
SCIENCE AND TECHNOLOGY                70
Name: topic, dtype: int64
(11,)
4734
9500
global                              9500
IRRELEVANT                          4734
MONEY MARKETS                       1673
SPORTS                              1102
FOREX MARKETS                        845
DEFENCE                              258
SHARE LISTINGS                       218
HEALTH                               183
BIOGRAPHIES PERSONALITIES PEOPLE     167
DOMESTIC MARKETS                     133
ARTS CULTURE ENTERTAINMENT           117
SCIENCE AND TECHNOLOGY                70
dtype: int64
25

In [ ]:
distr = {'global':{}}

for row in training_df.itertuples():
    sentence = row.article_words
    sentence = sentence.split(sep=',')
    words = len(sentence)
    if words not in distr['global']:
        distr['global'][words] = 0
    distr['global'][words] += 1
    #get label counts from dataframe
#     if 'count' not in distr['global']:
#         distr['global']['count'] = 0
#     distr['global']['count'] += 1
    if 'total' not in distr['global']:
        distr['global']['total'] = 0
    distr['global']['total'] += words
    #topic = row.topic.lower()
    topic = row.topic
    if topic not in distr:
        distr[topic] = {}
    if words not in distr[topic]:
        distr[topic][words] = 0
    distr[topic][words] += 1
    if 'total' not in distr[topic]:
        distr[topic]['total'] = 0
    distr[topic]['total'] += words

print(distr)

In [ ]:
##messy as hell
print('text length statistics')
for label, lengths in distr.items():
    num_samples = label_distr[label]
    avg = lengths['total']/num_samples
    del lengths['total'] #total should be separate...not in the distr data...classes might be easier
    filtered_lengths = [x for x in lengths.keys() if isinstance(x, int)]
    #print(filtered_lengths)
    deviation = std_dev(lengths, avg, num_samples)
    min_len = min(filtered_lengths)
    max_len = max(filtered_lengths)
    freqs = [v for k,v in lengths.items() if k != 'total']
    mode = max(freqs)
    modes = [k for k,v in lengths.items() if v == mode]
    coeff_variation = deviation/avg
    print(label, ":")
    print('\tavg: ', avg)
    print('\tstd_dev:', deviation)
    print('\tcoef_var:', coeff_variation)
    print('\tmin: ', min_len) 
    print('\tmax: ', max_len)
    print('\tmode: ', mode, modes)

In [ ]:
#corpus building
corpus = {}
for row in training_df.itertuples():
    words = row.article_words.split(',')
    for word in words:
        if word not in corpus:
            corpus[word] = {}
        if 'global' not in corpus[word]:
            corpus[word]['global'] = 0
        corpus[word]['global'] += 1
        if row.topic not in corpus[word]:
            corpus[word][row.topic] = 0
        corpus[word][row.topic] += 1
#print(corpus)

for word, word_dist in corpus.items():
    print(word)
    for topic, freq in word_dist.items():
        print('\t', topic, freq)

In [ ]:
##look into entropy and all that jazz and do some statistics on the corpus

In [17]:
class_corpus = {}
for row in training_df.itertuples():
    words = row.article_words.split(',')
    topic = row.topic
    if topic not in class_corpus:
        class_corpus[topic] = {}
    for word in words:
        if word not in class_corpus[topic]:
            class_corpus[topic][word] = 0
        class_corpus[topic][word] += 1

geom_class_corpus = {}
for topic, dist in class_corpus.items():
    geom_class_corpus[topic] = sorted(list(dist.items()), key=lambda x:-x[1])
    #geom_class_corpus[topic] = sorted(dist.items(), key=lambda x:x[1])

#print(list(geom_class_corpus['IRRELEVANT'].items()))

In [19]:
print('Conditional Distributions for each Topic')
k = 20
for topic, dist in geom_class_corpus.items():
    print(topic, dist[:k])

Conditional Distributions for each Topic
FOREX MARKETS [('dollar', 3358), ('bank', 2178), ('rate', 1755), ('trad', 1678), ('market', 1668), ('percent', 1336), ('currenc', 1292), ('deal', 1208), ('mark', 1191), ('yen', 962), ('cent', 953), ('clos', 932), ('high', 835), ('econom', 742), ('low', 675), ('year', 647), ('exchang', 635), ('foreign', 585), ('point', 558), ('end', 531)]
MONEY MARKETS [('bank', 4447), ('rate', 3772), ('percent', 3515), ('dollar', 3387), ('market', 2901), ('trad', 2490), ('day', 1775), ('deal', 1707), ('cent', 1533), ('currenc', 1472), ('mark', 1400), ('week', 1265), ('bill', 1246), ('clos', 1232), ('month', 1212), ('high', 1134), ('year', 1065), ('yen', 1057), ('econom', 1036), ('billion', 1012)]
SPORTS [('play', 1491), ('win', 1054), ('game', 947), ('year', 938), ('world', 928), ('match', 925), ('champ', 880), ('cup', 786), ('team', 734), ('scor', 707), ('final', 703), ('austral', 666), ('socc', 617), ('leagu', 609), ('point', 603), ('beat', 588), ('time', 539)